# <h1 align="center"> THEME 6 - Résolution analytique d'équations différentielles</h1>

### 🎯 Objectifs
- Résoudre des équations différentielles analytiquement

### 📚 Notions 

- [Exemple 1](#ex1): Réaction non-linéaire
- [Exemple 2](#ex2): Écoulement entre 2 plaques
- [Exemple 3](#ex3):

Un [lexique](#lexique) avec l'ensemble des fonctions qui ont été vues est disponible à la fin du notebook.

### 🧰 Librairies

- **Sympy**: est une librairie Python pour le calcul symbolique. Elle vise à devenir un système de calcul formel (CAS) complet tout en gardant le code aussi simple que possible afin d'être compréhensible et facilement extensible.

### 🔗 Référence

- [Documentation Sympy](https://docs.sympy.org/latest/reference/index.html#reference)

### ⚙️ Installation

`pip install sympy`

## <h2 align="center" id="ex1"> Exemple 1 - Réaction Non-Linéaire </h2>

### 📝 Contexte

Soit la réaction suivante:

$$
2 A \rightarrow B
$$

Son équation est une ODE non-linéaire:

$$
\frac{d C_{A}}{d t}=-k C_{A}^{2}
$$

### 🧪 Paramètres
 
Donnée:
- $k=2 \, L.mol^{-1}.min^{-1}$

Condition initiale:
- $C_{A0}=1 \, mol.L^{-1}$

### ⭐ Objectif

Résoudre l'ODE numériquement et tracer la concentration de A entre t=0 et t=10 min avec 26 points.

### 💻 Code

Sympy est une librairie très complète et qui possède un grand nombre de fonctions pour la résolution de toute sorte d'équations. Pour ce thème, le sous-module [ODE](https://docs.sympy.org/latest/modules/solvers/ode.html) de Sympy sera principalement utilisé.

Tout calcul symbolique avec Sympy se fait avec 3 types de données:
1. Des variables (ou symboles) définis avec `Symbol` ou `symbols`.
2. Des fonctions défénies avec `Function`.
3. Des équations (ou expressions) définies avec `Eq`.

Pour commencer, on définit notre équation non-linéaire sans utiliser de paramètres numériques. 

In [ ]:
import sympy as sp  # Importer la librairie sympy
import matplotlib.pyplot as plt
import numpy as np

sp.init_printing()  # Initialiser le systeme d'affichage LaTeX

# Déclarations
# ---------------------------------------------------
t, k = sp.symbols("t k")  # Variables t et k
C = sp.Function("C")  # Fonction C qui dépend de t
dCdt = sp.Derivative(C(t), t)
expr = sp.Eq(dCdt, -k * C(t) ** 2)
# Affichage de l'équation (uniquement dans un notebook)
# ---------------------------------------------------
expr

Une fois l'équation différentielle définie, on peut la résoudre avec la fonction `dsolve`.

In [ ]:
sol_analytique = sp.dsolve(expr)
sol_analytique

On peut remplacer une variable avec valeur numérique avec `subs`.

In [ ]:
sol_analytique.subs(k, 2)

Pour résoudre l'équation différentielle avec des conditions initiales, on peut passer un dictionnaire à l'argument `ics` de la fonction `dsolve`.

In [ ]:
k = 2  # Définir la valeur de k
expr = sp.Eq(dCdt, -k * C(t) ** 2)  # Redéfinir l'équation avec la nouvelle valeur de k

# ics prend un dictionnaire avec comme clé la fonction avec sa variable dépendante définie numériquement et comme valeur
# la condition initiale
sol_condinit = sp.dsolve(expr, ics={C(0): 1})  # C_A0 = 1
sol_condinit

On peut évaluer notre solution à un instant t en remplaçant la variable `t` par sa valeur numérique. 

In [ ]:
sol_condinit.subs(t, 3)

Noter que la solution est une équation, c'est à dire qu'elle contient une partie `lhs` (left-hand-side) dépendante de la variable d'état (ici `t`) et une partie `rhs` (right-hand-side) indépendante de cette variable.

Pour pouvoir évaluer la solution sur un ensemble de valeurs de `t` (comme un vecteur Numpy) il faut la transformer en fonction. Pour cela on utilise la fonction `lambdify` sur la partie `rhs` de la solution.

In [ ]:
from sympy.utilities.lambdify import lambdify

# Création d'une fonction qui prend en paramètre t et renvoie la valeur de la solution
f = lambdify(t, sol_condinit.rhs)
vec_t = np.linspace(0, 10, 100)

# La fonction peut être directement évaluée sur un vecteur Numpy
plt.plot(vec_t, f(vec_t))
plt.show()

## <h2 align="center" id="ex2"> Exemple 2 - Écoulement entre 2 plaques </h2>

### 📝 Contexte

Soit un fluide Newtonien s'écoulant entre deux plaques horizontales de longueur $L$ et séparées par un espace vertical de $2H$. L'axe $z$ est dans la direction de l'écoulement, et la vitesse du fluide dépend de la position verticale entre les deux plaques (l'axe $x$): $v_z(x)$. On opère en régime permanent, et on néglige les effets de bouts. C'est une différence de pression ($p_1 - p_0$) aux extrémités des plaques qui force cet écoulement laminaire. Les propriétés du fluide sont présumées constantes.

Les équations de Navier-Stokes se simplifient pour décrire le mouvement:

$$
\frac{d}{d z} p{\left(z \right)} = \mu \frac{d^{2}}{d x^{2}} \operatorname{v_{z}}{\left(x \right)}
$$

La température variera suivant $x$ et dépendra de la conduction et de la dissipation visqueuse comme suit:

$$
0 = k \frac{d^{2}}{d x^{2}} T{\left(x \right)} + \mu \left(\frac{d}{d x} \operatorname{v_{z}}{\left(x \right)}\right)^{2}
$$

On remarque que l'équation de mouvement est une EDO exacte, et que donc:

$$\frac{d^{}}{d z^{}} p{\left(z \right)} = \text{constante} $$

Que l'on peut reformuler comme:

$$\frac{d^{2}}{d z^{2}} p{\left(z \right)} = 0 $$

### 🧪 Paramètres

**Conditions frontalières (C.F.):**

Aux deux bouts de la plaque on a des pressions données:

$$
\begin{aligned}
\text{C.F.1} &: \quad \operatorname{p}{\left(z=0 \right)} = p_0 \\
\text{C.F.2} &: \quad \operatorname{p}{\left(z=L \right)} = p_1
\end{aligned}
$$


On présume qu'il n'y a pas de glissement, le fluide a donc une vitesse nulle au contact des plaques:

$$
\begin{aligned}
\text{C.F.3} &: \quad  \operatorname{v_z}{\left(x=H \right)} = 0 \\
\text{C.F.4} &: \quad \operatorname{v_z}{\left(x=-H \right)} = 0
\end{aligned}
$$

La première plaque (à $x=H$) est maintenue à une température $T_1$. 

$$  \text{C.F.5:} \quad T \left(x=H \right) = T_1 $$

L'autre plaque (à $x=-H$) est parfaitement isolée, donc sans échange de chaleur, $q_x$:

$$  \text{C.F.6:} \quad q_x \left(x=-H \right) = -k \frac{d^{2}}{d x^{2}} T{\left(x \right)}\vert_{x=H} = 0 $$

### ⭐ Objectif

Trouver une expression pour le profil de vitesse et de température du fluide.

### 💻 Code

#### Initialisation

On commence par définir les variables, équations et expressions que l'on va utiliser.

In [ ]:
# Déclarations
# ---------------------------------------------------
# Variables des équations
(
    k,
    x,
    z,
    mu,
) = sp.symbols("k, x, z, mu")
# Variables des conditions frontalières
T1, H, p0, p1, L = sp.symbols("T_1, H, p_0, p_1, L")
# Fonctions
vz = sp.Function("v_z")
p = sp.Function("p")
T = sp.Function("T")

In [ ]:
eq_mouvement = sp.Eq(p(z).diff(z), mu * vz(x).diff(x, 2))
eq_mouvement

In [ ]:
eq_energie = sp.Eq(0, k * T(x).diff(x, 2) + mu * (vz(x).diff(x)) ** 2)
eq_energie

In [ ]:
eq_pression = sp.Eq(0, p(z).diff(z, 2))
eq_pression

#### Résolution

La résolution de systèmes d'ODE dans Sympy est encore très expérimentale et ne supporte pas beaucoup de cas. Pour le moment, il est donc mieux de résoudre notre système en résolvant chacunes des équations séparemment et en injectant l'expression trouvée dans l'équation suivante. 

On résout l'équation de pression avec les conditions frontalières, ce qui nous donne un gradient de pression constant:

In [ ]:
# Résolution de l'équation de pression
sol_p = sp.dsolve(eq_pression, ics={p(0): p0, p(L): p1})
sol_p

Pour substituer facilement l'expression de $p(z)$ dans l'équation de mouvement, on créé un dictionnaire qui va contenir les expression des équations résolues. Ce dictionnaire est ensuite passé à `subs` avant.

In [ ]:
sol_dict = {}
sol_dict[p(z)] = sol_p.rhs
# Résolution de l'équation de mouvement
sol_m = sp.dsolve(eq_mouvement.subs(sol_dict), ics={vz(H): 0, vz(-H): 0})
sol_m

Finalement on effectue un substitution similaire pour résoudre l'équation d'énergie.

In [ ]:
sol_dict[vz(x)] = sol_m.rhs
# Résolution de l'équation d'énergie
sol_e = sp.dsolve(eq_energie.subs(sol_dict), ics={T(H): T1, T(x).diff(x).subs(x, -H): 0})
sol_dict[T(x)] = sol_e.rhs
sol_e

On remarque que l'expression pour $T(x)$ est assez complexe. Sympy met a disposition plusieurs fonctions pour manipuler l'expression d'une expression. Les fonctions les plus importantes sont:

- `.simplify()` pour simplifier l'expression
- `.expand()` pour développer l'expression
- `.factor()` pour factoriser l'expression

Ici on peut par exemple voir si l'expression de $T(x)-T_1$ pourrait être simplifiée et factorisée.

In [ ]:
# Simplification de l'expression de T(x) - T1
delta_T = (sol_dict[T(x)] - T1).simplify().factor()
delta_T

Pour pouvoir insérer nos résultats dans un document, la fonction `sp.print_latex()` permet d'afficher le code $LaTeX$ de notre expression.

In [ ]:
sp.print_latex(delta_T)  # Imprimer le code latex

#### Visualisation

En définissant des valeurs numériques à nos variables on peut visualiser le profil de vitesse et de température du fluide.

In [ ]:
# Définition des valeurs numériques pour chaque variable sympy
# ---------------------------------------------------
values = {H: 0.01, p0: 300e3, p1: 100e3, L: 10, k: 0.285, mu: 0.89, T1: 300}  # m  # Pa  # Pa  # m  # W/m.K  # Pa.s

f_T = sp.lambdify(x, sol_dict[T(x)].subs(values))
f_vz = sp.lambdify(x, sol_dict[vz(x)].subs(values))
vec_x = np.linspace(-values[H], values[H], 100)

In [ ]:
plt.xlabel("Position en x, de -H à H [m]")
plt.ylabel("Température [K]")
plt.plot(vec_x, f_T(vec_x))
plt.title("Profil de température dans le conduit")
plt.grid(True)
plt.show()

In [ ]:
plt.xlabel("Position en x, de -H à H [m]")
plt.ylabel("Vitesse d'écoulement ($v_z$) [m/s]")
plt.plot(vec_x, f_vz(vec_x))
plt.title("Profil de vitesse dans le conduit")
plt.grid(True)
plt.show()

## <h2 align="center" id="ex3"> Lexique </h2>

### 📚 Terminologie

### ✔️ Vu dans l'exemple 1